<a href="https://colab.research.google.com/github/chuahwb/FNB-Imagery-AI-Tool/blob/main/notebooks/mllm_image_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/chuahwb/FNB-Imagery-AI-Tool.git

Cloning into 'FNB-Imagery-AI-Tool'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 36 (delta 12), reused 4 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 52.55 KiB | 2.92 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [ ]:
# -*- coding: utf-8 -*-
"""
IPython Notebook for Phase 2: Evaluating Multimodal LLMs for F&B Image Recreation

This notebook connects to OpenRouter, processes local images, sends them with
prompts to selected multimodal LLMs, retrieves structured descriptions
using the 'instructor' library, tracks token usage, and estimates costs.
"""

# @title Setup: Install Libraries and Import Modules
# Install necessary libraries

"\nIPython Notebook for Phase 2: Evaluating Multimodal LLMs for F&B Image Recreation\n\nThis notebook connects to OpenRouter, processes local images, sends them with\nprompts to selected multimodal LLMs, retrieves structured descriptions\nusing the 'instructor' library, tracks token usage, and estimates costs.\n"

In [ ]:
!pip install instructor openai python-dotenv pillow pandas tqdm Jinja2 -q # Added Jinja2 for HTML templating

In [ ]:
import os
import base64
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field, field_validator, PrivateAttr
from PIL import Image
from io import BytesIO
from typing import List, Optional, Tuple, Dict, Any
import pandas as pd
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import time
import traceback # For detailed error logging
from jinja2 import Environment, FileSystemLoader, select_autoescape # For HTML report
import html # For escaping text in HTML

In [ ]:
# @title Configure API Key and OpenRouter Client

# --- IMPORTANT ---
# Set your OpenRouter API key.
# Option 1: Create a .env file in the same directory as this notebook
#           with the line: OPENROUTER_API_KEY="your-key-here"
# Option 2: Set it as an environment variable in your system.
# Option 3: Replace os.getenv("OPENROUTER_API_KEY") below with your actual key string
#           (less secure, not recommended for shared notebooks).
load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

if not OPENROUTER_API_KEY:
    print("⚠️ OpenRouter API Key not found.")
    print("Please set the OPENROUTER_API_KEY environment variable or in a .env file.")
    # You might want to raise an error or use input() here in a real script
    # OPENROUTER_API_KEY = input("Enter your OpenRouter API Key: ")


# Configure the Instructor client to use OpenRouter
# Patch the OpenAI client to add structured response capabilities
# Store the original unpatched client for accessing raw response data if needed
# Note: Instructor v1+ modifies the client in place. We access usage from the returned pydantic model's _raw_response attribute.
client = instructor.patch(
    OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENROUTER_API_KEY,
        default_headers={ # Optional, but good practice for OpenRouter
            "HTTP-Referer": "http://localhost:8888", # Replace with your app URL if deployed
            "X-Title": "F&B Image Eval", # Replace with your app name
        },
        timeout=600 # Increase timeout for potentially long image processing
    ),
    mode=instructor.Mode.MD_JSON # Use Markdown JSON mode for better compatibility
)

print("✅ OpenAI client patched with Instructor and configured for OpenRouter.")

✅ OpenAI client patched with Instructor and configured for OpenRouter.


In [ ]:
import os
from google.colab import drive

def load_images_from_drive(dataset_path):
  """Loads images from Google Drive and returns a list of tuples.

  Args:
    dataset_path: The path to the dataset folder in Google Drive.

  Returns:
    A list of tuples, where each tuple contains the image ID, path, and category.
  """

  drive.mount('/content/drive')
  images_data = []
  category_counts = {}

  for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
      for image_file in os.listdir(category_path):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
          image_id = os.path.splitext(image_file)[0]  # Use filename as ID
          image_path = os.path.join(category_path, image_file)
          images_data.append((image_id, image_path, category))
          category_counts[category] = category_counts.get(category, 0) + 1

  total_images = len(images_data)
  print(f"Total images loaded: {total_images}")
  print("Images loaded per category:")
  for category, count in category_counts.items():
    print(f"- {category}: {count}")

  return images_data

# Set the path to your dataset folder in Google Drive
dataset_path = '/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset'

# Load the images
IMAGES_TO_PROCESS = load_images_from_drive(dataset_path)

# Now IMAGES_TO_PROCESS contains your list of tuples
# You can use it in your existing code

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total images loaded: 2
Images loaded per category:
- Product Shot: 1
- Location Ambience Shots: 1


In [ ]:
# @title Define Pydantic Model for Structured Description
# This model mirrors the 8 points requested in the prompts

class FnbImageDescription(BaseModel):
    """Structured description of an F&B social media image."""
    primary_subject: str = Field(..., description="Detailed description of the main food, drink, person, or element, including ingredients, preparation, presentation, and actions.")
    composition_framing: str = Field(..., description="Description of layout (e.g., centered, rule of thirds), camera angle (e.g., eye-level, overhead), and framing (e.g., close-up, medium shot).")
    background_environment: str = Field(..., description="Details of the setting, surfaces, other objects, and depth of field (e.g., blurred background).")
    lighting_color: str = Field(..., description="Description of light source, style (e.g., natural, studio), direction, shadows, highlights, dominant colors, and temperature.")
    texture_materials: str = Field(..., description="Specific textures visible (e.g., glossy sauce, crispy batter, smooth ceramic, condensation).")
    text_branding: str = Field(..., description="Accurate transcription of visible text and detailed description of logos or branding elements.")
    mood_atmosphere: str = Field(..., description="Overall feeling conveyed by the image (e.g., cozy, vibrant, elegant, casual).")
    overall_style: str = Field(..., description="Characterization of the image style (e.g., photorealistic, cinematic, flat lay, illustration).")

    # Store raw response for usage data access using PrivateAttr for internal use
    # This avoids the Pydantic field naming conflict.
    _raw_response: Optional[Any] = PrivateAttr(default=None)

    # Optional: Add a validator to ensure fields are not empty
    @field_validator('*', mode='before')
    def check_not_empty(cls, value):
        # This validator applies to the main fields, not the private attribute
        if isinstance(value, str) and not value.strip():
            return "(Not specified)" # Provide a default if empty
        return value

print("✅ Pydantic model 'FnbImageDescription' defined.")

# @title Define Image Handling Function

def encode_image_to_base64(image_path: str, max_size=(1024, 1024)) -> Optional[str]:
    """Loads an image, resizes if needed, and encodes it to base64."""
    try:
        with Image.open(image_path) as img:
            # Convert image to RGB if it's not (e.g., RGBA, P)
            if img.mode != 'RGB':
                img = img.convert('RGB')

            # Optional: Resize image to prevent exceeding token limits
            # Uncomment the line below if images are very large
            # print(f"    Original size: {img.size}")
            # img.thumbnail(max_size, Image.Resampling.LANCZOS)
            # print(f"    Resized to: {img.size}")

            buffered = BytesIO()
            img.save(buffered, format="JPEG", quality=85) # Save as JPEG with quality setting
            img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
            # print(f"    Encoded Base64 length: {len(img_str)}") # For debugging size
            return img_str
    except FileNotFoundError:
        print(f"❌ Error: Image file not found at {image_path}")
        return None
    except Exception as e:
        print(f"❌ Error encoding image {image_path}: {e}")
        return None

print("✅ Image encoding function 'encode_image_to_base64' defined.")

# @title Define Prompt Construction Function

# Baseline Prompt (as defined previously)
BASELINE_PROMPT = """
Analyze the provided F&B image in meticulous detail. Generate a comprehensive description suitable for recreating this exact image using a text-to-image AI. Describe the following elements:

1.  **Primary Subject(s):** Identify and describe the main food, drink, person, or element. Include details like ingredients, preparation style (e.g., grilled, fried, steamed), presentation, specific actions (e.g., pouring, eating).
2.  **Composition & Framing:** Describe the layout (e.g., centered, rule of thirds, asymmetrical), camera angle (e.g., eye-level, overhead shot, low angle, Dutch tilt), and framing (e.g., extreme close-up, close-up, medium shot, full shot, wide shot).
3.  **Background & Environment:** Detail the setting (e.g., restaurant table, kitchen counter, outdoor picnic, abstract background), surfaces (e.g., wooden table, marble countertop, checkered tablecloth), other objects present (e.g., cutlery, napkins, other dishes, decor), and depth of field (e.g., sharp focus on subject with heavily blurred background, deep focus with everything sharp).
4.  **Lighting & Color:** Describe the light source and style (e.g., bright natural daylight from window, warm indoor ambient light, dramatic studio flash, soft diffused light), direction of light, presence and softness of shadows, highlights, dominant color palette, and overall color temperature (e.g., warm tones, cool tones, vibrant, muted).
5.  **Texture & Materials:** Mention specific textures visible (e.g., glossy sauce, crispy batter, fluffy bread, smooth ceramic plate, rough wooden board, condensation on glass, metallic sheen of cutlery).
6.  **Text & Branding:** Accurately transcribe any visible text (e.g., on packaging, menus, signs). Describe any logos, specific brand colors used prominently, or recognizable branding elements in detail.
7.  **Mood & Atmosphere:** Describe the overall feeling conveyed by the image (e.g., cozy and intimate, bright and energetic, rustic and homely, elegant and sophisticated, casual and fun, busy and dynamic).
8.  **Overall Style:** Characterize the image style (e.g., photorealistic, cinematic, food photography style, candid shot, flat lay, vector illustration, graphic design with photo elements).
"""

# Category-Specific Emphasis (as defined previously)
CATEGORY_EMPHASIS = {
    "Product Shot": "Emphasis for Product Shot: Pay extra attention to the details of the food/drink item itself – texture, color accuracy, freshness indicators (e.g., steam, droplets), plating details, garnishes, and how the lighting highlights the product's appeal. Describe the dishware/glassware precisely.",
    "Lifestyle Shot": "Emphasis for Lifestyle Shot: Focus on the people involved – their expressions, actions, interactions with the product or each other, clothing style, and body language. Describe how the product is integrated into the scene and the overall narrative suggested (e.g., friends enjoying brunch, family dinner, solo coffee break).",
    "Menu Displays": "Emphasis for Menu Display: Prioritize accurate transcription of all visible text, including item names, descriptions, and prices. Describe the menu's layout, typography (font style, size, weight), color scheme, any graphical elements (lines, boxes, icons), and the material/context if it's a physical menu photo (e.g., chalkboard, printed paper, digital screen). Note the overall readability and design style.",
    "Promotional Graphics": "Emphasis for Promotional Graphic: Accurately transcribe all promotional text (offer details, dates, calls to action). Describe the graphic design elements used (e.g., background color/gradient, shapes, icons, font styles). If it combines photos and graphics, describe how they are integrated. Detail the overall visual hierarchy and intended message.",
    "Branding Elements": "Emphasis for Branding Element: Focus intensely on the specific branding element shown (e.g., logo, packaging detail, unique sign). Describe its colors, shapes, typography, and material precisely. Explain its context within the image and how it contributes to the overall brand identity.",
    "Location/Ambience Shots": "Emphasis for Location/Ambience: Describe the key features of the space – decor style (e.g., modern, rustic, industrial), furniture, lighting fixtures, color scheme, materials (wood, brick, metal), sense of space (cozy, spacious), cleanliness, and overall atmosphere it creates for a customer. Mention specific details like wall art, plants, table settings if visible.",
    "Event Promotions": "Emphasis for Event Promotion: Accurately transcribe all event details (name, date, time, location, description, contact info, price). Describe any specific imagery related to the event theme (e.g., musical instruments, wine bottles, specific food). Detail the overall design style of the flyer/poster/graphic and its call to action.",
    "Behind-the-Scenes (BTS)": "Emphasis for BTS: Describe the action taking place (e.g., cooking, plating, ingredient prep, staff interaction). Detail the environment (e.g., kitchen equipment, staff uniforms, raw ingredients) and the sense of activity or focus. Capture the candid, authentic feel typical of BTS shots.",
    "Default": "" # For categories not listed or if no emphasis is needed
}

def construct_prompt(category: Optional[str] = None, use_category_emphasis: bool = False) -> str:
    """Constructs the prompt, optionally adding category-specific emphasis."""
    prompt = BASELINE_PROMPT
    if use_category_emphasis and category:
        emphasis = CATEGORY_EMPHASIS.get(category, CATEGORY_EMPHASIS["Default"])
        if emphasis:
            prompt += "\n\n" + emphasis
    return prompt

print("✅ Prompt construction function 'construct_prompt' defined.")

# @title Define Core Inference Function (with Token Usage)

def get_structured_description_with_usage(
    model_name: str,
    image_base64: str,
    prompt: str
) -> Tuple[Optional[FnbImageDescription], Optional[Dict[str, int]]]:
    """
    Sends image and prompt to a model via OpenRouter, gets a structured description,
    and extracts token usage information.
    Returns a tuple: (description_object, usage_dict)
    """
    usage_info = None
    description_obj = None
    try:
        print(f"   Querying {model_name}...")
        start_time = time.time()

        # Make the API call requesting the Pydantic model response
        description_obj = client.chat.completions.create(
            model=model_name,
            response_model=FnbImageDescription,
            max_retries=0, # Retry once on failure
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{image_base64}",
                                # OpenAI API suggests 'detail' param, but OpenRouter might not use it
                                # explicitly. High detail is usually default for base64.
                                # "detail": "high"
                            },
                        },
                    ],
                }
            ],
            max_tokens=2048, # Adjust as needed
            temperature=0.2, # Lower temperature for more deterministic descriptions
        )
        end_time = time.time()
        print(f"   ✅ Success for {model_name} in {end_time - start_time:.2f} seconds.")

        # --- Extract Token Usage ---
        # Access the private attribute _raw_response correctly
        raw_response = getattr(description_obj, '_raw_response', None)
        if raw_response and hasattr(raw_response, 'usage'):
             raw_usage = raw_response.usage
             if raw_usage:
                 usage_info = {
                     "prompt_tokens": raw_usage.prompt_tokens,
                     "completion_tokens": raw_usage.completion_tokens,
                     "total_tokens": raw_usage.total_tokens,
                 }
                 # print(f"      Usage: {usage_info}") # Uncomment for debugging
        else:
             # Fallback if usage is not directly on the response object's attribute
             # This might happen depending on instructor/openai library versions
             # Try accessing from the raw response dict if possible
             try:
                 if description_obj and hasattr(description_obj.model_extra, 'usage'):
                     raw_usage = description_obj.model_extra['usage']
                     usage_info = {
                         "prompt_tokens": raw_usage.prompt_tokens,
                         "completion_tokens": raw_usage.completion_tokens,
                         "total_tokens": raw_usage.total_tokens,
                     }
                     # print(f"      Usage (fallback): {usage_info}")
                 else:
                    print(f"   ⚠️ Usage information not found in response for {model_name}.")
                    usage_info = {"prompt_tokens": 0, "completion_tokens": 0, "total_tokens": 0}
             except Exception:
                 print(f"   ⚠️ Error accessing fallback usage info for {model_name}.")
                 usage_info = {"prompt_tokens": 0, "completion_tokens": 0, "total_tokens": 0}


    except Exception as e:
        print(f"   ❌ Error querying {model_name}:")
        # Print detailed traceback for debugging
        # traceback.print_exc()
        print(f"      {e}")
        # Ensure description_obj is None if error occurs before assignment
        description_obj = None
        usage_info = {"prompt_tokens": 0, "completion_tokens": 0, "total_tokens": 0} # Default on error

    return description_obj, usage_info

print("✅ Core inference function 'get_structured_description_with_usage' defined.")


# @title Define Main Processing Workflow (with Cost Estimation)

# --- Configuration ---
# List of OpenRouter model identifiers to test (Update based on Step 1.2 and availability)
# Ensure these models support vision input on OpenRouter
MODELS_TO_TEST = [
    "openai/gpt-4o-2024-11-20",
    "openai/gpt-4.1-mini",
    #"openai/o4-mini": {"input_cost_per_M": 1.10, "output_cost_per_M": 4.40},
    "openai/o3",
    # Anthropic - Claude
    "anthropic/claude-3.7-sonnet",
    # Google - Gemini
    "google/gemini-2.5-pro-preview-03-25",
    "google/gemini-2.5-flash-preview",
    # Meta - Llama
    "meta-llama/llama-4-scout:free",
    "meta-llama/llama-4-maverick:free",
    "meta-llama/llama-3.2-90b-vision-instruct",
    # XAI - Grok
    "x-ai/grok-3-beta",
    # Qwen
    "qwen/qwen2.5-vl-32b-instruct",
]

# --- !!! IMPORTANT: Define Model Pricing (per Million Tokens) !!! ---
# GET THESE VALUES FROM https://openrouter.ai/models FOR ACCURACY
# Prices are in USD per 1 Million tokens (Input and Output)
MODEL_PRICING = {
    # Model Identifier: {"input_cost_per_M": X.XX, "output_cost_per_M": Y.YY}
    # OpenAI
    #"openai/gpt-4o": {"input_cost_per_M": 5.00, "output_cost_per_M": 15.00},
    "openai/gpt-4o-2024-11-20": {"input_cost_per_M": 2.50, "output_cost_per_M": 10.00},
    "openai/gpt-4.1-mini": {"input_cost_per_M": 0.40, "output_cost_per_M": 1.60},
    #"openai/o4-mini": {"input_cost_per_M": 1.10, "output_cost_per_M": 4.40},
    "openai/o3": {"input_cost_per_M": 10.00, "output_cost_per_M": 40.00},
    # Anthropic - Claude
    "anthropic/claude-3.7-sonnet": {"input_cost_per_M": 3.00, "output_cost_per_M": 15.00},
    # Google - Gemini
    "google/gemini-2.5-pro-preview-03-25": {"input_cost_per_M": 1.25, "output_cost_per_M": 10.00},
    "google/gemini-2.5-flash-preview": {"input_cost_per_M": 0.15, "output_cost_per_M": 0.6},
    # Meta - Llama
    "meta-llama/llama-4-scout:free": {"input_cost_per_M": 0.0, "output_cost_per_M": 0.00},
    "meta-llama/llama-4-maverick:free": {"input_cost_per_M": 0.0, "output_cost_per_M": 0.00},
    "meta-llama/llama-3.2-90b-vision-instruct": {"input_cost_per_M": 0.8, "output_cost_per_M": 1.60},
    # XAI - Grok
    "x-ai/grok-3-beta": {"input_cost_per_M": 3.0, "output_cost_per_M": 15.00},
    # Qwen
    "qwen/qwen2.5-vl-32b-instruct": {"input_cost_per_M": 0.90, "output_cost_per_M": 0.90},
}
print("⚠️ Ensure MODEL_PRICING dictionary is updated with current OpenRouter prices!")


# --- Input Data ---
# List of images to process. Each item is a tuple: (image_id, image_path, category)
# Replace with your actual image paths and categories from Step 1.1
# IMAGES_TO_PROCESS = [
#     ("prod_001", "path/to/your/product_shot_1.jpg", "Product Shot"),
#     ("life_001", "path/to/your/lifestyle_shot_1.png", "Lifestyle Shot"),
#     ("menu_001", "path/to/your/menu_display_1.jpg", "Menu Displays"),
#     ("promo_001", "path/to/your/promo_graphic_1.jpeg", "Promotional Graphics"),
#     # Add all other images from your dataset here...
# ]

✅ Pydantic model 'FnbImageDescription' defined.
✅ Image encoding function 'encode_image_to_base64' defined.
✅ Prompt construction function 'construct_prompt' defined.
✅ Core inference function 'get_structured_description_with_usage' defined.
⚠️ Ensure MODEL_PRICING dictionary is updated with current OpenRouter prices!


In [ ]:
# --- Workflow Execution ---

results_list = []

# Use tqdm for progress bar
for image_id, image_path, category in tqdm(IMAGES_TO_PROCESS, desc="Processing Images"):
    print(f"\nProcessing Image: {image_id} ({category}) - {image_path}")

    # 1. Encode Image
    image_base64 = encode_image_to_base64(image_path)
    if not image_base64:
        print(f"   Skipping image {image_id} due to encoding error.")
        continue

    # 2. Construct Prompt (Choose whether to use category emphasis)
    # Set use_category_emphasis=True to add specific instructions
    use_category_emphasis_flag = False # Or True
    prompt_text = construct_prompt(category, use_category_emphasis=use_category_emphasis_flag)

    # 3. Iterate through models
    for model_name in tqdm(MODELS_TO_TEST, desc=f"  Models for {image_id}", leave=False):
        description_obj, usage_info = get_structured_description_with_usage(model_name, image_base64, prompt_text)

        # --- Calculate Estimated Cost ---
        estimated_cost = 0.0
        prompt_tokens = 0
        completion_tokens = 0
        total_tokens = 0

        if usage_info:
            prompt_tokens = usage_info.get("prompt_tokens", 0)
            completion_tokens = usage_info.get("completion_tokens", 0)
            total_tokens = usage_info.get("total_tokens", 0)

            if model_name in MODEL_PRICING:
                pricing = MODEL_PRICING[model_name]
                input_cost = (prompt_tokens / 1_000_000) * pricing["input_cost_per_M"]
                output_cost = (completion_tokens / 1_000_000) * pricing["output_cost_per_M"]
                estimated_cost = input_cost + output_cost
            else:
                print(f"   ⚠️ Pricing not found for model {model_name}. Cost estimation skipped.")

        # Store results
        result_data = {
            "Image ID": image_id,
            "Image Path": image_path,
            "Category": category,
            "Model": model_name,
            "Prompt Type": "Category-Specific" if use_category_emphasis_flag and category else "Baseline",
            "Prompt Tokens": prompt_tokens,
            "Completion Tokens": completion_tokens,
            "Total Tokens": total_tokens,
            "Estimated Cost (USD)": round(estimated_cost, 6) # Round to 6 decimal places
        }

        if description_obj:
            # Add structured fields to the result dictionary
            # Use model_dump() which correctly handles Pydantic models without private attributes
            result_data.update(description_obj.model_dump())
            result_data["Status"] = "Success"
        else:
            # Add empty fields if the description failed
            # Iterate through the model's defined fields
            for field_name in FnbImageDescription.model_fields:
                 result_data[field_name] = "ERROR"
            result_data["Status"] = "Error"

        results_list.append(result_data)

print("\n✅ Workflow finished.")

# @title Display Results in a DataFrame

# Convert results to DataFrame early for easier processing
results_df = pd.DataFrame() # Initialize empty DataFrame
if results_list:
    results_df = pd.DataFrame(results_list)

    # Define column order for better readability
    display_column_order = [
        "Image ID", "Category", "Model", "Status", "Prompt Type",
        "Prompt Tokens", "Completion Tokens", "Total Tokens", "Estimated Cost (USD)",
        # Add the description fields
        "primary_subject", "composition_framing", "background_environment",
        "lighting_color", "texture_materials", "text_branding",
        "mood_atmosphere", "overall_style",
        "Image Path" # Include image path for reference
    ]
    # Ensure all expected columns exist before reordering
    results_df = results_df.reindex(columns=display_column_order, fill_value=None)


    # Set display options for better readability
    pd.set_option('display.max_rows', 50) # Adjust as needed
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000) # Adjust for wider terminal/output
    pd.set_option('display.max_colwidth', 100) # Adjust width as needed

    print("\n--- Comparison Results (DataFrame) ---")
    # Display the DataFrame
    display(results_df)

    # --- Optional: Calculate and Display Summary Statistics ---
    print("\n--- Summary Statistics ---")
    # Total cost
    total_estimated_cost = results_df["Estimated Cost (USD)"].sum()
    print(f"Total Estimated Cost: ${total_estimated_cost:.6f}")

    # Average cost per model
    avg_cost_per_model = results_df.groupby("Model")["Estimated Cost (USD)"].mean()
    print("\nAverage Estimated Cost per Model:")
    display(avg_cost_per_model)

    # Average tokens per model
    avg_tokens_per_model = results_df.groupby("Model")[["Prompt Tokens", "Completion Tokens", "Total Tokens"]].mean()
    print("\nAverage Tokens per Model:")
    display(avg_tokens_per_model)


    # --- Optional: Save results to CSV ---
    # try:
    #     results_df.to_csv("fnb_llm_evaluation_results_with_cost.csv", index=False)
    #     print("\n✅ Results saved to fnb_llm_evaluation_results_with_cost.csv")
    # except Exception as e:
    #     print(f"\n❌ Error saving results to CSV: {e}")

else:
    print("\nNo results generated.")


Processing Images:   0%|          | 0/2 [00:00<?, ?it/s]


Processing Image: prod_0001 (Product Shot) - /content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset/Product Shot/prod_0001.jpeg


  Models for prod_0001:   0%|          | 0/11 [00:00<?, ?it/s]

   Querying openai/gpt-4o-2024-11-20...
   ❌ Error querying openai/gpt-4o-2024-11-20:
      Error code: 402 - {'error': {'message': 'Insufficient credits. Add more using https://openrouter.ai/settings/credits', 'code': 402}}
   Querying openai/gpt-4.1-mini...
   ❌ Error querying openai/gpt-4.1-mini:
      Error code: 402 - {'error': {'message': 'Insufficient credits. Add more using https://openrouter.ai/settings/credits', 'code': 402}}
   Querying openai/o3...
   ❌ Error querying openai/o3:
      Error code: 403 - {'error': {'message': 'OpenAI is requiring a key to access this model, which you can add in https://openrouter.ai/settings/integrations - you can also switch to o3-mini.', 'code': 403}}
   Querying anthropic/claude-3.7-sonnet...
   ❌ Error querying anthropic/claude-3.7-sonnet:
      Error code: 402 - {'error': {'message': 'Insufficient credits. Add more using https://openrouter.ai/settings/credits', 'code': 402}}
   Querying google/gemini-2.5-pro-preview-03-25...
   ❌ Error q

  Models for loc_0001:   0%|          | 0/11 [00:00<?, ?it/s]

...

,Image ID,Category,Model,Status,Prompt Type,Prompt Tokens,Completion Tokens,Total Tokens,Estimated Cost (USD),primary_subject,composition_framing,background_environment,lighting_color,texture_materials,text_branding,mood_atmosphere,overall_style,Image Path
0,prod_0001,Product Shot,openai/gpt-4o-2024-11-20,Error,Baseline,0,0,0,0.0,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset/Product Shot/prod_0001.jpeg
1,prod_0001,Product Shot,openai/gpt-4.1-mini,Error,Baseline,0,0,0,0.0,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset/Product Shot/prod_0001.jpeg
2,prod_0001,Product Shot,openai/o3,Error,Baseline,0,0,0,0.0,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset/Product Shot/prod_0001.jpeg
3,prod_0001,Product Shot,anthropic/claude-3.7-sonnet,Error,Baseline,0,0,0,0.0,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset/Product Shot/prod_0001.jpeg
4,prod_0001,Product Shot,google/gemini-2.5-pro-preview-03-25,Error,Baseline,0,0,0,0.0,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset/Product Shot/prod_0001.jpeg
5,prod_0001,Product Shot,google/gemini-2.5-flash-preview,Error,Baseline,0,0,0,0.0,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset/Product Shot/prod_0001.jpeg
6,prod_0001,Product Shot,meta-llama/llama-4-scout:free,Success,Baseline,2093,592,2685,0.0,"The main subject of the image is a bottle of Absolut Vodka, specifically the 'Absolut Country of...","The composition is centered, with the bottle placed in the middle of the image. The framing is a...","The background of the image is a vibrant, abstract environment featuring a mix of dark blues and...","The lighting in the image is bright and vivid, with a dominant color palette that includes shade...","The image features a variety of textures, including the smooth, glossy surface of the vodka bott...","The visible text includes the brand name 'ABSOLUT' in large purple letters, accompanied by the p...","The overall mood of the image is playful, imaginative, and vibrant. The whimsical illustrations ...","The image style is a form of graphic design or vector illustration, characterized by bold lines,...",/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset/Product Shot/prod_0001.jpeg
7,prod_0001,Product Shot,meta-llama/llama-4-maverick:free,Success,Baseline,2093,502,2595,0.0,"A bottle of Absolut Vodka, specifically the 'Country of Sweden Classic' variant, prominently dis...","The image is centered around the vodka bottle, with the bottle taking up a significant portion o...","The background is an abstract, illustrative environment featuring swirling patterns, clouds, and...",The lighting in the image is illustrative and not based on a real-world light source. The color ...,"The image features a variety of textures, primarily due to its illustrative style. The bottle ap...",The text 'ABSOLUT Country of Sweden CLASSIC' is prominently displayed on the label of the bottle...,"The overall mood of the image is vibrant, playful, and imaginative. The use of bright colors and...","The image is a vector illustration with a graphic design style that incorporates detailed, color...",/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset/Product Shot/prod_0001.jpeg
8,prod_0001,Product Shot,meta-llama/llama-3.2-90b-vision-instruct,Error,Baseline,0,0,0,0.0,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset/Product Shot/prod_0001.jpeg
9,prod_0001,Product Shot,x-ai/grok-3-beta,Error,Baseline,0,0,0,0.0,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,/content/drive/MyDrive/AI Imagery Marke


--- Summary Statistics ---
Total Estimated Cost: $0.000000

Average Estimated Cost per Model:


,Estimated Cost (USD)
Model,
anthropic/claude-3.7-sonnet,0.0
google/gemini-2.5-flash-preview,0.0
google/gemini-2.5-pro-preview-03-25,0.0
meta-llama/llama-3.2-90b-vision-instruct,0.0
meta-llama/llama-4-maverick:free,0.0
meta-llama/llama-4-scout:free,0.0
openai/gpt-4.1-mini,0.0
openai/gpt-4o-2024-11-20,0.0
openai/o3,0.0



Average Tokens per Model:


,Prompt Tokens,Completion Tokens,Total Tokens
Model,,,
anthropic/claude-3.7-sonnet,0.0,0.0,0.0
google/gemini-2.5-flash-preview,0.0,0.0,0.0
google/gemini-2.5-pro-preview-03-25,0.0,0.0,0.0
meta-llama/llama-3.2-90b-vision-instruct,0.0,0.0,0.0
meta-llama/llama-4-maverick:free,2528.0,428.0,2956.0
meta-llama/llama-4-scout:free,2528.0,499.5,3027.5
openai/gpt-4.1-mini,0.0,0.0,0.0
openai/gpt-4o-2024-11-20,0.0,0.0,0.0
openai/o3,0.0,0.0,0.0


In [ ]:

# @title Generate HTML Report for Visual Evaluation

# --- Configuration for HTML Report ---
HTML_REPORT_FILENAME = "fnb_evaluation_report.html"
# Set this path if your images are in a specific folder relative to the notebook
# or use absolute paths in IMAGES_TO_PROCESS. Leave as "" if paths in
# IMAGES_TO_PROCESS are already correct relative to the HTML file location.
# IMAGE_BASE_PATH_FOR_HTML = "images/" # Example: "images/" or "/path/to/images/"
IMAGE_BASE_PATH_FOR_HTML = "" # Assume paths in df are correct relative to HTML

# Define the HTML template using Jinja2 syntax in a string
html_template_string = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>F&B Image Evaluation Report</title>
    <style>
        body { font-family: sans-serif; margin: 20px; line-height: 1.6; }
        .image-section { margin-bottom: 40px; border-bottom: 2px solid #eee; padding-bottom: 20px; }
        .image-container img { max-width: 400px; max-height: 400px; border: 1px solid #ccc; margin-bottom: 15px; }
        table { border-collapse: collapse; width: 100%; margin-top: 15px; font-size: 0.9em; }
        th, td { border: 1px solid #ddd; padding: 8px; text-align: left; vertical-align: top; }
        th { background-color: #f2f2f2; }
        tr:nth-child(even) { background-color: #f9f9f9; }
        td:first-child { font-weight: bold; min-width: 150px; } /* Model name column */
        .status-success { color: green; }
        .status-error { color: red; font-weight: bold; }
        .description-cell { white-space: pre-wrap; word-wrap: break-word; } /* Wrap long text */
    </style>
</head>
<body>
    <h1>F&B Image Evaluation Report</h1>
    <p>Date Generated: {{ generation_date }}</p>

    {% for image_id, group in results.groupby('Image ID') %}
    <div class="image-section">
        <h2>Image ID: {{ image_id }}</h2>
        <p><strong>Category:</strong> {{ group['Category'].iloc[0] }}</p>
        <div class="image-container">
            {% set img_path = image_base_path + group['Image Path'].iloc[0] %}
            <img src="{{ img_path }}" alt="Image {{ image_id }}" onerror="this.alt='Image not found at {{ img_path }}'; this.style.border='1px solid red';">
        </div>

        <table>
            <thead>
                <tr>
                    <th>Model</th>
                    <th>Status</th>
                    <th>Total Tokens</th>
                    <th>Est. Cost (USD)</th>
                    <th>Primary Subject</th>
                    <th>Composition/Framing</th>
                    <th>Background/Environment</th>
                    <th>Lighting/Color</th>
                    <th>Texture/Materials</th>
                    <th>Text/Branding</th>
                    <th>Mood/Atmosphere</th>
                    <th>Overall Style</th>
                </tr>
            </thead>
            <tbody>
                {% for index, row in group.iterrows() %}
                <tr>
                    <td>{{ row['Model'] }}</td>
                    <td class="status-{{ row['Status'].lower() }}">{{ row['Status'] }}</td>
                    <td>{{ row['Total Tokens'] }}</td>
                    <td>{{ "%.6f"|format(row['Estimated Cost (USD)']) }}</td>
                    <td class="description-cell">{{ escape(row['primary_subject']) }}</td>
                    <td class="description-cell">{{ escape(row['composition_framing']) }}</td>
                    <td class="description-cell">{{ escape(row['background_environment']) }}</td>
                    <td class="description-cell">{{ escape(row['lighting_color']) }}</td>
                    <td class="description-cell">{{ escape(row['texture_materials']) }}</td>
                    <td class="description-cell">{{ escape(row['text_branding']) }}</td>
                    <td class="description-cell">{{ escape(row['mood_atmosphere']) }}</td>
                    <td class="description-cell">{{ escape(row['overall_style']) }}</td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
    </div>
    {% endfor %}

</body>
</html>
"""

def generate_html_report(df, output_filename, image_base_path=""):
    """Generates an HTML report from the results DataFrame."""
    if df.empty:
        print("❌ Cannot generate report: Results DataFrame is empty.")
        return

    # Ensure necessary columns exist, especially 'Image Path'
    if 'Image Path' not in df.columns:
        print("❌ Cannot generate report: 'Image Path' column missing in results.")
        return

    try:
        # Using Jinja2 directly with the template string
        env = Environment(loader=None, autoescape=select_autoescape(['html', 'xml']))
        env.globals['escape'] = html.escape # Add escape function to template context
        template = env.from_string(html_template_string)

        generation_date = time.strftime("%Y-%m-%d %H:%M:%S %Z")

        # Render the template
        html_content = template.render(
            results=df,
            generation_date=generation_date,
            image_base_path=image_base_path
        )

        # Write to file
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(html_content)
        print(f"✅ HTML report generated successfully: {output_filename}")

    except Exception as e:
        print(f"❌ Error generating HTML report: {e}")
        traceback.print_exc()

# --- Generate the Report ---
# Make sure results_df exists and is populated from the previous cell
if 'results_df' in locals() and not results_df.empty:
     generate_html_report(results_df, HTML_REPORT_FILENAME, IMAGE_BASE_PATH_FOR_HTML)
elif not results_list:
     print("⚠️ Skipping HTML report generation because no results were generated in the workflow.")
else:
     print("⚠️ Skipping HTML report generation. Ensure the main workflow cell has been run successfully and 'results_df' exists.")


from IPython.display import HTML

with open('fnb_evaluation_report.html', 'r') as f:  # Assuming your file is named 'fnb_evaluation_report.html'
  html_content = f.read()

display(HTML(html_content))


✅ HTML report generated successfully: fnb_evaluation_report.html


Model,Status,Total Tokens,Est. Cost (USD),Primary Subject,Composition/Framing,Background/Environment,Lighting/Color,Texture/Materials,Text/Branding,Mood/Atmosphere,Overall Style
openai/gpt-4o-2024-11-20,Error,0,0.000000,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR
openai/gpt-4.1-mini,Error,0,0.000000,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR
openai/o3,Error,0,0.000000,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR
anthropic/claude-3.7-sonnet,Error,0,0.000000,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR
google/gemini-2.5-pro-preview-03-25,Error,0,0.000000,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR
google/gemini-2.5-flash-preview,Error,0,0.000000,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR
meta-llama/llama-4-scout:free,Success,3370,0.000000,"The primary subject of the image is a dimly lit restaurant interior with several patrons seated at tables. The main elements are the dining area, the people, and the decorative mural on the wall.","The image is framed with a medium shot, capturing the restaurant&#x27;s interior. The composition is asymmetrical, with a large tree trunk and a mural on the wall drawing the viewer&#x27;s attention. The camera angle appears to be at eye-level, providing a natural perspective of the scene.","The background features a dark red wall with a large mural of a woman in a stylish outfit, surrounded by flowers. The setting includes a wooden floor, a wooden and marble table with chairs, and other patrons seated at different tables. The atmosphere is cozy with a touch of elegance.","The lighting is warm and dim, likely from indoor ambient light sources. The dominant color palette is warm tones, with soft shadows and highlights that add depth to the image. The overall color temperature is cozy and inviting.","The image features various textures, including the smooth marble tabletop, the wooden chairs and floor, and the rough tree trunk. The mural on the wall adds a touch of artistic texture, while the patrons&#x27; clothing and the decorative elements contribute to the overall visual texture.",There is no visible text or branding in the image that can be accurately transcribed or described.,"The overall mood of the image is cozy and intimate, with a touch of elegance. The dim lighting and warm colors create a welcoming atmosphere, suggesting a relaxed dining experience.","The image style is photorealistic, capturing the details of the restaurant interior and the patrons in a natural and unobtrusive manner. The cinematic aspect is evident in the composition and lighting, which create a visually appealing scene."
meta-llama/llama-4-maverick:free,Success,3317,0.000000,"A dimly lit restaurant interior with patrons seated at tables, a large mural of a woman on the wall, and a tree trunk in the background.","The image is framed as a medium shot, capturing the restaurant&#x27;s ambiance with a slightly low angle, emphasizing the ceiling and the mural. The composition is asymmetrical, with the tree trunk on one side and the mural on the other.","The setting is a restaurant with wooden floors, dark red walls, and a wooden ceiling. Tables are set with marble tops and wooden chairs. A large tree trunk is visible on the left, and a mural of a woman dominates the right wall. The depth of field is moderate, with the foreground and background elements visible but not sharply focused.","The lighting is warm and ambient, likely from overhead lights and possibly table lamps, creating soft shadows. The dominant color palette is warm, with tones of brown, red, and beige. The overall color temperature is warm.","Visible textures include the smooth marble table tops, the wooden chairs and floor, and the rough bark of the tree trunk. The mural adds a touch of artistic texture to the wall.",No specific text or branding elements are visible in the image.,"The overall feeling conveyed by the image is cozy and intimate, with a sense of warmth and comfort. The dim lighting and rich colors contribute to a sophistic